# Showcase: `isocor` Python package

In [ ]:
import isocor

The correction of a measurement vector starts by instantiating a metabolite-specific
*corrector* object and calling its `.correct()` method.

## Instantiation of a *corrector*

We recommend that you use `MetaboliteCorrectorFactory`
to choose the best correction strategy depending on your resolution:

In [ ]:
corrector = isocor.mscorrectors.MetaboliteCorrectorFactory("C3PO", tracer="13C")
corrector_HR = isocor.mscorrectors.MetaboliteCorrectorFactory("C3PO", tracer="13C", resolution=1e4, mz_of_resolution=400, charge=1)

In [ ]:
corrector

In [ ]:
corrector_HR

### Watch out for default parameters!

You should pay attention to the default parameters of `MetaboliteCorrectorFactory`,
such as the mass and natural abundance stored for each isotope,
the formula used to interpret the resolution, etc.

In [ ]:
corrector.data_isotopes

In [ ]:
corrector.tracer_purity  # perfect purity: [0% of 12C, 100% of 13C]

### Direct instantiation

If you don't want to use the factory or want to create your own *corrector*, you can use
`LowResMetaboliteCorrector` and
`HighResMetaboliteCorrector` directly for respectively low and high resolution data.

But keep in mind that we do not use the same algorithm at low and high resolution.

In [ ]:
corrector_LR  = isocor.mscorrectors.LowResMetaboliteCorrector(formula="C3PO", tracer="13C",
                                                              derivative_formula=None,
                                                              tracer_purity=[0.0, 1.0],
                                                              data_isotopes=isocor.mscorrectors.LowResMetaboliteCorrector.DEFAULT_ISODATA,
                                                              correct_NA_tracer= False)
corrector_LR

## Correction of a measurement vector

The correction is performed on a vector of measurements through the `.correct()` method:

In [ ]:
corrected_area, iso_fraction, res, m_enr = corrector.correct([0., 4000., 200., 0.])
corrected_area

In addition of the corrected area, `.correct()` returns:

* isotopologue_fraction, that is the abundance of each tracer isotopologue (corrected area normalized to 1)
* residuum
* mean enrichment

One corrector can safely be re-used for other measurements vectors (given that it is the same metabolite/derivative/parameters), for instance in another sample:

In [ ]:
corrector.correct([0., 4000, 2000, 1000])

Empty vectors will give: 

In [ ]:
corrector.correct([0., 0., 0., 0.])

IsoCor check the inputs you provide and warn you if you did something unexpected:

In [ ]:
try:
    corrector.correct([0., 0., 0.])  # our corrector was defined for a C3, so we expect 4 "peaks" (n+1)
except Exception as e:
    print(e.__class__, e)

## Readable attributes

Everything is accessible from the *corrector*.

In [ ]:
corrector.formula

The correction matrix is always available (computed on-the-fly at first access): 

In [ ]:
corrector.correction_matrix

In [ ]:
corrector_HR.correction_matrix

Of course, other attributes and parameters are available too:

In [ ]:
corrector.molecular_weight

In [ ]:
corrector.correct_NA_tracer

The correction matrix is unique to a corrector object. **You should never have to set or reset the correction matrix yourself**.

If you need a new correction matrix, you should re-instanciate a new corrector object.
IsoCor forbids you to change directly the correction matrix:

In [ ]:
try:
    corrector.correction_matrix = [45]
except Exception as e:
    print(e.__class__, e)

In the same vein, you should never change parameters of the corrector at runtime. IsoCor might even forbid you to do so:

In [ ]:
try:
    corrector.formula = {'H': 2, 'O': 1}
except Exception as e:
    print(e.__class__, e)

As a rule of thumbs, always instantiate a new corrector if anything else than the measurement vector changed in your analysis.

## Catch the logs

You can use *logging* to catch the logs. By default, logs go to the `stderr` (that is printed in Jupyter notebooks):

In [ ]:
import logging

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%H:%M', level=logging.DEBUG)

In [ ]:
# Let's do something silly
isocor.MetaboliteCorrectorFactory("C3PO", '13C', resolution=0.1, mz_of_resolution= 400, charge=1)

In [ ]:
# it is forbidden to half-define a isocorrector
try:
    isocor.MetaboliteCorrectorFactory("C3PO", '13C', mz_of_resolution= 400, charge=1)
except Exception as e:
    print(e.__class__, e)